In [1]:
import json
import time
import textwrap

In [2]:
import dotenv
dotenv.load_dotenv()
from langchain_openai import ChatOpenAI, AzureChatOpenAI

chat = AzureChatOpenAI(
    max_tokens=16000,
    temperature=0.5,
    deployment_name="gpt-4o-b",
    model_name="gpt-4o",
    # deployment_name="gpt-35-turbo",
    # model_name="gpt-3.5-turbo",
)
print(f'Using Azure endpoint: {chat.azure_endpoint}')

ModuleNotFoundError: No module named 'langchain_core.load.dump'

## Prompt

In [3]:
# Test if LLM responds OK
prompt = "What is your training data cutoff date?"
response = chat(prompt)
print(response.content)

/var/folders/py/lx68lx7s6sbcr2m9kwtspqvh0000gn/T/ipykernel_37363/597314759.py:3: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat(prompt)


My training data cutoff date is **October 2021**. This means I was trained on information that was available up until that point. If you're asking about events or developments after that date, I might not have the most accurate or up-to-date information. Let me know how I can assist you!


In [4]:
reply = "\n".join(textwrap.wrap(response.content))
print(reply)

My training data cutoff date is **October 2021**. This means I was
trained on information that was available up until that point. If
you're asking about events or developments after that date, I might
not have the most accurate or up-to-date information. Let me know how
I can assist you!


In [5]:
print(response.model_dump_json(indent=2))

{
  "content": "My training data cutoff date is **October 2021**. This means I was trained on information that was available up until that point. If you're asking about events or developments after that date, I might not have the most accurate or up-to-date information. Let me know how I can assist you!",
  "additional_kwargs": {
    "refusal": null
  },
  "response_metadata": {
    "token_usage": {
      "completion_tokens": 61,
      "prompt_tokens": 15,
      "total_tokens": 76,
      "completion_tokens_details": {
        "accepted_prediction_tokens": 0,
        "audio_tokens": 0,
        "reasoning_tokens": 0,
        "rejected_prediction_tokens": 0
      },
      "prompt_tokens_details": {
        "audio_tokens": 0,
        "cached_tokens": 0
      }
    },
    "model_name": "gpt-4o-2024-11-20",
    "system_fingerprint": "fp_82ce25c0d4",
    "prompt_filter_results": [
      {
        "prompt_index": 0,
        "content_filter_results": {
          "hate": {
            "filtered"

In [6]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [7]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [8]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [9]:
customer_style = """American English \
in a calm and respectful tone
"""

In [10]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [11]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [12]:
type(customer_messages)

list

In [13]:
print(customer_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone
. text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [14]:
customer_response = chat(customer_messages)
reply = "\n".join(textwrap.wrap(customer_response.content))
print(reply)

I’m feeling quite frustrated because the lid of my blender came off,
causing my smoothie to splatter all over the kitchen walls. To make
things even more difficult, the warranty doesn’t cover the cleanup
costs. I could really use your help with this situation.


## Parser

In [37]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON object with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [38]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON object with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [39]:
customer_messages = prompt_template.format_messages(text=customer_review)

In [40]:
customer_response = chat(customer_messages)
print(customer_response.content)

```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": [
    "It's slightly more expensive than the other leaf blowers out there",
    "I think it's worth it for the extra features"
  ]
}
```


In [41]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [42]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")

delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [43]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [44]:
format_instructions = output_parser.get_format_instructions()

In [45]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [46]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [47]:
customer_response = chat(customer_messages)
print(customer_response.content)

```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": [
    "It's slightly more expensive than the other leaf blowers out there",
    "I think it's worth it for the extra features"
  ]
}
```


In [48]:
type(customer_response.content)

str

In [49]:
output_dict = output_parser.parse(customer_response.content)

In [50]:
type(output_dict)

dict